In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import json
import os
import re
import pandas as pd
import numpy as np

import time

from bs4 import BeautifulSoup 
import urllib
import requests
import html5lib
import json
import traceback

In [4]:
bddf = pd.read_json(open(os.path.join('./data', 'info_wo_audio.json')), orient='index')
bddf.head()

,name,sci_name,id,data-xc-filepath,data-xc-id,page,row,length,contributor,date,time,country,location,elev,type,note,bird_seen,playback_used,class
0,Melodious Warbler,Hippolais-polyglotta,xc_audio_121277_573,//xeno-canto.org/sounds/uploaded/UVCHPWXJBN/XC...,121277,0,6,1:05,Tomas Belka,2012-07-06,07:00,Albania,"Memoraq, Sarandë, Vlora",180,alarm call,2 adults with food in the beak.,yes,no,A
1,Houbara Bustard,Chlamydotis-undulata,xc_audio_644134_581,//xeno-canto.org/sounds/uploaded/XYWBPFBYXQ/XC...,644134,0,14,0:07,Larbi Badaoui,2021-04-21,06:30,Algeria,"Naama, Naama, Naâma Province",1191,"alarm call, bird in hand, juvenile, sex uncert...",Recording (not its ID) has been discussed. See...,NaN,NaN,A
2,Algerian Nuthatch,Sitta-ledanti,xc_audio_454880_585,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454880,0,18,0:53,Andrew Spencer,2018-12-20,10:50,Algeria,"Djimla, Jijel Province",550,song,Songs from the same male as in XC454879. Indi...,yes,yes,A
3,Algerian Nuthatch,Sitta-ledanti,xc_audio_454879_586,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454879,0,19,1:06,Andrew Spencer,2018-12-20,10:50,Algeria,"Djimla, Jijel Province",550,call,Calls from a male bird moving very high in tal...,yes,yes,A
4,Algerian Nuthatch,Sitta-ledanti,xc_audio_454878_587,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454878,0,20,4:33,Andrew Spencer,2018-12-20,10:00,Algeria,"Djimla, Jijel Province",550,song,Songs from the same individual as in XC454877....,yes,yes,A


In [5]:
dfa = bddf[bddf['class']=='A']
dfa.head()

,name,sci_name,id,data-xc-filepath,data-xc-id,page,row,length,contributor,date,time,country,location,elev,type,note,bird_seen,playback_used,class
0,Melodious Warbler,Hippolais-polyglotta,xc_audio_121277_573,//xeno-canto.org/sounds/uploaded/UVCHPWXJBN/XC...,121277,0,6,1:05,Tomas Belka,2012-07-06,07:00,Albania,"Memoraq, Sarandë, Vlora",180,alarm call,2 adults with food in the beak.,yes,no,A
1,Houbara Bustard,Chlamydotis-undulata,xc_audio_644134_581,//xeno-canto.org/sounds/uploaded/XYWBPFBYXQ/XC...,644134,0,14,0:07,Larbi Badaoui,2021-04-21,06:30,Algeria,"Naama, Naama, Naâma Province",1191,"alarm call, bird in hand, juvenile, sex uncert...",Recording (not its ID) has been discussed. See...,NaN,NaN,A
2,Algerian Nuthatch,Sitta-ledanti,xc_audio_454880_585,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454880,0,18,0:53,Andrew Spencer,2018-12-20,10:50,Algeria,"Djimla, Jijel Province",550,song,Songs from the same male as in XC454879. Indi...,yes,yes,A
3,Algerian Nuthatch,Sitta-ledanti,xc_audio_454879_586,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454879,0,19,1:06,Andrew Spencer,2018-12-20,10:50,Algeria,"Djimla, Jijel Province",550,call,Calls from a male bird moving very high in tal...,yes,yes,A
4,Algerian Nuthatch,Sitta-ledanti,xc_audio_454878_587,//xeno-canto.org/sounds/uploaded/CDTGHVBGZP/XC...,454878,0,20,4:33,Andrew Spencer,2018-12-20,10:00,Algeria,"Djimla, Jijel Province",550,song,Songs from the same individual as in XC454877....,yes,yes,A


In [6]:
mfdf = dfa['name'].value_counts()
mfdf120 = mfdf[mfdf > 120]
mfdf120.index

Index(['Rufous-bellied Thrush', 'Yellow-olive Flatbill'], dtype='object')

In [7]:
ndf = dfa[dfa['name'].isin(mfdf120.index)]
print(ndf.shape)
ndf.reset_index(inplace=True)
ndf.rename(columns={'index': 'key'}, inplace=True)
ndf.head()

(250, 19)


,key,name,sci_name,id,data-xc-filepath,data-xc-id,page,row,length,contributor,date,time,country,location,elev,type,note,bird_seen,playback_used,class
0,167,Rufous-bellied Thrush,Turdus-rufiventris,xc_audio_695280_550,//xeno-canto.org/sounds/uploaded/XTVEPHMPPJ/XC...,695280,48,11,4:34,Niels Krabbe,2022-01-03,06:11,Argentina,Salta: La Misión,1270,song,bird-seen:no,no,no,A
1,180,Rufous-bellied Thrush,Turdus-rufiventris,xc_audio_692377_568,//xeno-canto.org/sounds/uploaded/XTVEPHMPPJ/XC...,692377,48,29,1:38,Niels Krabbe,2021-12-21,06:06,Argentina,"Salta: Vaqueros, 12 km N Salta",1270,song,bird-seen:no,no,no,A
2,188,Rufous-bellied Thrush,Turdus-rufiventris,xc_audio_690795_80,//xeno-canto.org/sounds/uploaded/XTVEPHMPPJ/XC...,690795,49,7,0:15,Niels Krabbe,2021-12-10,06:43,Argentina,"Salta: Gallinato, 1370 m",1360,song,Just a single phrase before the bird took off.,yes,no,A
3,308,Rufous-bellied Thrush,Turdus-rufiventris,xc_audio_671301_646,//xeno-canto.org/sounds/uploaded/VLDFGFKOWN/XC...,671301,53,10,1:38,Franco Vushurovich,2018-12-02,06:00,Argentina,"PP Urugua-i, Misiones",320,song,bird-seen:no,no,no,A
4,362,Rufous-bellied Thrush,Turdus-rufiventris,xc_audio_648127_477,//xeno-canto.org/sounds/uploaded/VLDFGFKOWN/XC...,648127,55,5,0:09,Franco Vushurovich,2018-04-19,13:30,Argentina,"Pilcomayo, Formosa",80,"begging call, subsong",bird-seen:no,no,no,A


In [ ]:
ids = ndf['id'].unique()
assert len(ids) == 250

In [14]:
path_selected_sound = os.path.join('./data', 'sound_selected_db')
for ri, row in ndf.iterrows():
    time.sleep(np.random.randint(700)/1000.)
    with open(os.path.join(path_selected_sound, f"sound_{row['key']}.mp3"), "wb") as f_out:
        f_out.write(requests.get(f"http:{row['data-xc-filepath']}").content)
